In [2]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Step 2: Read the file
import pandas as pd

# Set file path
file_path = '/content/drive/My Drive/IDS 575 Final project/crimes data.csv'

# Read the CSV file and parse date columns
df = pd.read_csv(file_path, parse_dates=['Date', 'Updated On'])

# Display the first few rows to confirm successful reading
df.head()

<ipython-input-3-89fe39381447>:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_path, parse_dates=['Date', 'Updated On'])
<ipython-input-3-89fe39381447>:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_path, parse_dates=['Date', 'Updated On'])


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,13707986,JJ100019,2024-12-31 23:45:00,117XX S STATE ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,9.0,53.0,08B,1178352.0,1827293.0,2024,2025-01-08 15:42:09,41.681396,-87.622767,"(41.68139574, -87.622767037)"
1,13707849,JJ100011,2024-12-31 23:45:00,018XX W MAYPOLE AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,False,...,27.0,28.0,14,1164081.0,1901067.0,2024,2025-01-08 15:42:09,41.884152,-87.672933,"(41.884152322, -87.672932576)"
2,13707847,JJ100007,2024-12-31 23:42:00,029XX W CHICAGO AVE,1345,CRIMINAL DAMAGE,TO CITY OF CHICAGO PROPERTY,CTA BUS,False,False,...,36.0,24.0,14,1156857.0,1905197.0,2024,2025-01-08 15:42:09,41.895635,-87.699348,"(41.895634912, -87.699347915)"
3,13707836,JJ100034,2024-12-31 23:40:00,0000X S OAKLEY BLVD,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,27.0,28.0,07,1161090.0,1899804.0,2024,2025-01-08 15:42:09,41.880749,-87.683951,"(41.880749175, -87.683950956)"
4,13709517,JJ101458,2024-12-31 23:34:00,047XX W HARRISON ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,24.0,25.0,14,1144914.0,1896933.0,2024,2025-01-08 15:42:09,41.873191,-87.743421,"(41.873191313, -87.743420863)"


In [8]:
# Step 3: Check for missing values in each column
missing = df.isnull().sum()
print("Missing values per column:\n", missing)

# Fields to pay special attention to:
important_fields = ['Date', 'Primary Type', 'Community Area', 'Latitude', 'Longitude', 'Arrest']
print("Missing values in key fields:\n", df[important_fields].isnull().sum())

Missing values per column:
 ID                         0
Case Number                0
Date                       0
Block                      0
IUCR                       0
Primary Type               0
Description                0
Location Description    3405
Arrest                     0
Domestic                   0
Beat                       0
District                   0
Ward                      13
Community Area             2
FBI Code                   0
X Coordinate            6485
Y Coordinate            6485
Year                       0
Updated On                 0
Latitude                6485
Longitude               6485
Location                6485
dtype: int64
Missing values in key fields:
 Date                 0
Primary Type         0
Community Area       2
Latitude          6485
Longitude         6485
Arrest               0
dtype: int64


In [10]:
# Step 4: Confirm that dates fall between 2022-01-01 and 2024-12-31
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # Force conversion to datetime type
date_min = df['Date'].min()
date_max = df['Date'].max()

print(f"Earliest date in the dataset: {date_min}")
print(f"Latest date in the dataset: {date_max}")

# Check for records outside the date range
mask = (df['Date'] < '2022-01-01') | (df['Date'] > '2024-12-31')
print(f"Number of records outside the date range: {df[mask].shape[0]}")

Earliest date in the dataset: 2022-01-01 00:00:00
Latest date in the dataset: 2024-12-31 23:45:00
Number of records outside the date range: 553


In [11]:
# Step 5: Check data types of each column
print("Column data types:")
print(df.dtypes)

# Ensure 'Arrest' is of boolean type; convert if not
if df['Arrest'].dtype != 'bool':
    df['Arrest'] = df['Arrest'].astype(bool)
    print("Column 'Arrest' has been converted to boolean.")

Column data types:
ID                               int64
Case Number                     object
Date                    datetime64[ns]
Block                           object
IUCR                            object
Primary Type                    object
Description                     object
Location Description            object
Arrest                            bool
Domestic                          bool
Beat                             int64
District                         int64
Ward                           float64
Community Area                 float64
FBI Code                        object
X Coordinate                   float64
Y Coordinate                   float64
Year                             int64
Updated On              datetime64[ns]
Latitude                       float64
Longitude                      float64
Location                        object
dtype: object


In [12]:
# Step 6: Check for missing or invalid latitude/longitude values
invalid_lat = df['Latitude'].isnull() | (df['Latitude'] < 40) | (df['Latitude'] > 43)
invalid_lon = df['Longitude'].isnull() | (df['Longitude'] < -88) | (df['Longitude'] > -87)

print(f"Number of invalid Latitude records: {invalid_lat.sum()}")
print(f"Number of invalid Longitude records: {invalid_lon.sum()}")

Number of invalid Latitude records: 6487
Number of invalid Longitude records: 6487


In [13]:
# Step 7: Check how many unique Community Areas there are
print("Number of unique Community Areas:", df['Community Area'].nunique())
print("List of unique Community Area IDs:", sorted(df['Community Area'].dropna().unique()))

Number of unique Community Areas: 77
List of unique Community Area IDs: [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(6.0), np.float64(7.0), np.float64(8.0), np.float64(9.0), np.float64(10.0), np.float64(11.0), np.float64(12.0), np.float64(13.0), np.float64(14.0), np.float64(15.0), np.float64(16.0), np.float64(17.0), np.float64(18.0), np.float64(19.0), np.float64(20.0), np.float64(21.0), np.float64(22.0), np.float64(23.0), np.float64(24.0), np.float64(25.0), np.float64(26.0), np.float64(27.0), np.float64(28.0), np.float64(29.0), np.float64(30.0), np.float64(31.0), np.float64(32.0), np.float64(33.0), np.float64(34.0), np.float64(35.0), np.float64(36.0), np.float64(37.0), np.float64(38.0), np.float64(39.0), np.float64(40.0), np.float64(41.0), np.float64(42.0), np.float64(43.0), np.float64(44.0), np.float64(45.0), np.float64(46.0), np.float64(47.0), np.float64(48.0), np.float64(49.0), np.float64(50.0), np.float64(51.0), np.float64(52.0), 

In [ ]:
# Keep only data within the specified date range

In [14]:
df = df[(df['Date'] >= '2022-01-01') & (df['Date'] <= '2024-12-31')]

In [ ]:
# Remove records with missing Community Area values

In [15]:
df = df[df['Community Area'].notnull()]
df['Community Area'] = df['Community Area'].astype(int)

<ipython-input-15-205353014c67>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Community Area'] = df['Community Area'].astype(int)


In [ ]:
# Remove records with missing or invalid latitude/longitude

In [16]:
df = df[
    (df['Latitude'].notnull()) & (df['Longitude'].notnull()) &
    (df['Latitude'] >= 40) & (df['Latitude'] <= 43) &
    (df['Longitude'] >= -88) & (df['Longitude'] <= -87)
]

In [17]:
print(f"Current number of records: {len(df)}")

Current number of records: 753001


In [18]:
print(f"Earliest date: {df['Date'].min()}")
print(f"Latest date: {df['Date'].max()}")

Earliest date: 2022-01-01 00:00:00
Latest date: 2024-12-31 00:00:00


In [19]:
print("Number of abnormal Latitude values:", ((df['Latitude'] < 40) | (df['Latitude'] > 43)).sum())
print("Number of abnormal Longitude values:", ((df['Longitude'] < -88) | (df['Longitude'] > -87)).sum())

Number of abnormal Latitude values: 0
Number of abnormal Longitude values: 0


In [20]:
print("Minimum Community Area value:", df['Community Area'].min())
print("Maximum Community Area value:", df['Community Area'].max())
print("Number of unique Community Areas:", df['Community Area'].nunique())

Minimum Community Area value: 1
Maximum Community Area value: 77
Number of unique Community Areas: 77


In [21]:
# Set new save path
save_path = '/content/drive/My Drive/IDS 575 Final project/cleaned_crimes_data.csv'

# Save as a new CSV file
df.to_csv(save_path, index=False)

# Confirm successful save
print(f"File has been saved to: {save_path}")

File has been saved to: /content/drive/My Drive/IDS 575 Final project/cleaned_crimes_data.csv
